# Import Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('../data/raw_data.csv',encoding="ISO-8859-1")
data.head()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_29184\468009671.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../data/raw_data.csv',encoding="ISO-8859-1")


,stn_code,sampling_date,state,location,agency,type,so2,no2,rspm,spm,location_monitoring_station,pm2_5,date
0,150.0,February - M021990,Andhra Pradesh,Hyderabad,NaN,"Residential, Rural and other Areas",4.8,17.4,NaN,NaN,NaN,NaN,1990-02-01
1,151.0,February - M021990,Andhra Pradesh,Hyderabad,NaN,Industrial Area,3.1,7.0,NaN,NaN,NaN,NaN,1990-02-01
2,152.0,February - M021990,Andhra Pradesh,Hyderabad,NaN,"Residential, Rural and other Areas",6.2,28.5,NaN,NaN,NaN,NaN,1990-02-01
3,150.0,March - M031990,Andhra Pradesh,Hyderabad,NaN,"Residential, Rural and other Areas",6.3,14.7,NaN,NaN,NaN,NaN,1990-03-01
4,151.0,March - M031990,Andhra Pradesh,Hyderabad,NaN,Industrial Area,4.7,7.5,NaN,NaN,NaN,NaN,1990-03-01


# Clean Data

In [3]:
data.drop(['stn_code', 'sampling_date', 'agency','location_monitoring_station'], axis=1, inplace=True)
data.reset_index(drop=True)

,state,location,type,so2,no2,rspm,spm,pm2_5,date
0,Andhra Pradesh,Hyderabad,"Residential, Rural and other Areas",4.8,17.4,NaN,NaN,NaN,1990-02-01
1,Andhra Pradesh,Hyderabad,Industrial Area,3.1,7.0,NaN,NaN,NaN,1990-02-01
2,Andhra Pradesh,Hyderabad,"Residential, Rural and other Areas",6.2,28.5,NaN,NaN,NaN,1990-02-01
3,Andhra Pradesh,Hyderabad,"Residential, Rural and other Areas",6.3,14.7,NaN,NaN,NaN,1990-03-01
4,Andhra Pradesh,Hyderabad,Industrial Area,4.7,7.5,NaN,NaN,NaN,1990-03-01
...,...,...,...,...,...,...,...,...,...
435737,West Bengal,ULUBERIA,RIRUO,22.0,50.0,143.0,NaN,NaN,2015-12-24
435738,West Bengal,ULUBERIA,RIRUO,20.0,46.0,171.0,NaN,NaN,2015-12-29
435739,andaman-and-nicobar-islands,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
435740,Lakshadweep,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
data["type"].unique()

array(['Residential, Rural and other Areas', 'Industrial Area', nan,
       'Sensitive Area', 'Industrial Areas', 'Residential and others',
       'Sensitive Areas', 'Industrial', 'Residential', 'RIRUO',
       'Sensitive'], dtype=object)

In [5]:
cleaning_map = {
    'Residential, Rural and other Areas': 'Residential',
    'Residential and others': 'Residential',
    'RIRUO': 'Residential',
    'Industrial Area': 'Industrial',
    'Industrial Areas': 'Industrial',
    'Sensitive Area': 'Sensitive',
    'Sensitive Areas': 'Sensitive'
}

data['type'] = data['type'].replace(cleaning_map)

print(data)

                              state   location         type   so2   no2  \
0                    Andhra Pradesh  Hyderabad  Residential   4.8  17.4   
1                    Andhra Pradesh  Hyderabad   Industrial   3.1   7.0   
2                    Andhra Pradesh  Hyderabad  Residential   6.2  28.5   
3                    Andhra Pradesh  Hyderabad  Residential   6.3  14.7   
4                    Andhra Pradesh  Hyderabad   Industrial   4.7   7.5   
...                             ...        ...          ...   ...   ...   
435737                  West Bengal   ULUBERIA  Residential  22.0  50.0   
435738                  West Bengal   ULUBERIA  Residential  20.0  46.0   
435739  andaman-and-nicobar-islands        NaN          NaN   NaN   NaN   
435740                  Lakshadweep        NaN          NaN   NaN   NaN   
435741                      Tripura        NaN          NaN   NaN   NaN   

         rspm  spm  pm2_5        date  
0         NaN  NaN    NaN  1990-02-01  
1         NaN  NaN 

In [6]:
data.drop(index=[435739,435740,435741],inplace=True)

In [7]:
columns_to_fill = ['so2', 'no2','rspm','spm','pm2_5']  # Specify the columns you want to fill

# Fill NaN values in the specified columns with their respective means
for column in columns_to_fill:
    data[column] = data[column].fillna(data[column].mean())

In [8]:
data

,state,location,type,so2,no2,rspm,spm,pm2_5,date
0,Andhra Pradesh,Hyderabad,Residential,4.8,17.4,108.832784,220.78348,40.791467,1990-02-01
1,Andhra Pradesh,Hyderabad,Industrial,3.1,7.0,108.832784,220.78348,40.791467,1990-02-01
2,Andhra Pradesh,Hyderabad,Residential,6.2,28.5,108.832784,220.78348,40.791467,1990-02-01
3,Andhra Pradesh,Hyderabad,Residential,6.3,14.7,108.832784,220.78348,40.791467,1990-03-01
4,Andhra Pradesh,Hyderabad,Industrial,4.7,7.5,108.832784,220.78348,40.791467,1990-03-01
...,...,...,...,...,...,...,...,...,...
435734,West Bengal,ULUBERIA,Residential,20.0,44.0,148.000000,220.78348,40.791467,2015-12-15
435735,West Bengal,ULUBERIA,Residential,17.0,44.0,131.000000,220.78348,40.791467,2015-12-18
435736,West Bengal,ULUBERIA,Residential,18.0,45.0,140.000000,220.78348,40.791467,2015-12-21
435737,West Bengal,ULUBERIA,Residential,22.0,50.0,143.000000,220.78348,40.791467,2015-12-24


In [9]:
data.dropna(inplace=True)
data.head()
data.reset_index(drop=True)

,state,location,type,so2,no2,rspm,spm,pm2_5,date
0,Andhra Pradesh,Hyderabad,Residential,4.8,17.4,108.832784,220.78348,40.791467,1990-02-01
1,Andhra Pradesh,Hyderabad,Industrial,3.1,7.0,108.832784,220.78348,40.791467,1990-02-01
2,Andhra Pradesh,Hyderabad,Residential,6.2,28.5,108.832784,220.78348,40.791467,1990-02-01
3,Andhra Pradesh,Hyderabad,Residential,6.3,14.7,108.832784,220.78348,40.791467,1990-03-01
4,Andhra Pradesh,Hyderabad,Industrial,4.7,7.5,108.832784,220.78348,40.791467,1990-03-01
...,...,...,...,...,...,...,...,...,...
430340,West Bengal,ULUBERIA,Residential,20.0,44.0,148.000000,220.78348,40.791467,2015-12-15
430341,West Bengal,ULUBERIA,Residential,17.0,44.0,131.000000,220.78348,40.791467,2015-12-18
430342,West Bengal,ULUBERIA,Residential,18.0,45.0,140.000000,220.78348,40.791467,2015-12-21
430343,West Bengal,ULUBERIA,Residential,22.0,50.0,143.000000,220.78348,40.791467,2015-12-24


In [ ]:
data.to_csv("../data/clean_data.csv")

: 